**Task:** 

Given a protein (see attached PDB file) and a set of docked ligand poses (attached SDF file), could you please write a script that filters out ligand poses that do __not__ contain a salt bridge between the ligand 5HT2A and the Aspartate at index 155 in the PDB file. The skeleton of the main function is provided. You are free to write as many helper functions as you wish. You may use any Python based chemoinformatic library (RDKit, OpenEye, OpenBabel).


__Hints:__

- We define a salt bridge as an oxygen-nitrogen with a distance less than 3.5Å. Conjugated nitrogens and nitrogens adjacent to aromatic systems should be discluded. 


- In this task you will: 
    - load the protein and ligands
    - select atoms that could potentially form a salt bridge at D155
    - compute the distance between them
    - filter out ligand poses that do not form a salt bridge
    - saves the remiaining poses to an SDF file.
    
__Bonus points for:__

- Writing all functions with docstrings and specifying the types of the inputs and outputs.
- Adding an SDTag to the filtered ligands, indicating the distance of the salt bridge.

### Step 1: Import the necessary libraries

In [1]:
from Bio.PDB import PDBParser
import Bio.PDB.Atom
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

### Step 2: Define helper functions to check aromaticity of Nitrogen's neighbours and compute distance to detect salt bridge:

In [2]:
def is_aromatic_neighbour(atom):
    """
    Summary: this function checks if a nitrogen is adjacent to an aromatic ring
    
    Args:
        atom: str
            The mol object for the nitrogen atom
    
    Returns:
        True or False
    """
    #get neighbouring atoms:
    neighbors = atom.GetNeighbors()
    
    #check if those are aromatic:
    aromatic_neighbors = any(neighbor.GetIsAromatic() for neighbor in neighbors)
                    
    if aromatic_neighbors:
        return True
    else:
        return False

def is_salt_bridge(res, ligand, salt_bridge_distance_cutoff = 3.5):
    """
    Summary: This function checks if a ligand forms a salt bridge with ASP155 based on a distance cutoff. 
    It iterates over the OD1 and OD2 atoms of the Aspartate residue and the N and O atoms of the ligand, and calculates the distance 
    between them. If the distance is less than the cutoff, it returns True, indicating a potential salt bridge.
    
    Args:
        res: int
            This is the target residue ID: in this case ASP155
        
        ligand: str
            The mol object the represents the docked pose
            
        salt_bridge_distance_cutoff: float
            Docked poses will be kept only if they have a salt bridge less than this specified value.
    
    Returns:
        (distance and True) OR (None and False)
    """
    
    conf = ligand.GetConformer()
    
    # Enumerate through both side-chain oxygens of ASP155 and nitrogen and oxygen of ligand poses:
    for atom1 in res.get_list():
        if atom1.get_name() in ["OD1", "OD2"]:
            for atom2 in ligand.GetAtoms():
                if atom2.GetSymbol() in ["N"] and not atom2.IsInRing() and not is_aromatic_neighbour(atom2): #make sure that the nitrogen is NOT part of the ring
                    
                    # Get the XYZ coordinates
                    position1 = atom1.get_coord()
                    position2 = conf.GetAtomPosition(atom2.GetIdx())
                    
                    #Use the Euclidean distance formula to calculate the distance between the two atoms:
                    distance = ((position1[0] - position2.x) ** 2 +
                                    (position1[1] - position2.y) ** 2 +
                                    (position1[2] - position2.z) ** 2) ** 0.5
                    
                    if distance < salt_bridge_distance_cutoff:
                        return distance, True
    else:                
        return None, False

### Step 3: Define the main function to filter ligand poses depending upon presence or absence of salt bridge

In [3]:
def filter_ligand_poses(pdb_file, sdf_file, output_file, residue_position=155):
    """
    Summary: This function filters out ligand poses that do not form a salt bridge with the Aspartate residue at the specified 155 position.
    
    Args:
        pdb_file: str
            The PDB file of the protein to analyse.
            
        sdf_file: str
            The input SDF file of the docked ligand poses to filter.
            
        output_file: str
            The sdf file to write the filtered ligands to.
            
        residue_position: int
            The residue index representing the conserved Aspartate. It is 155 for the task by default.
    
    Returns:
        None 
    """
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    model = structure[0]

    # Enumerate through all amino acids in the sequence to find if ASP is indeed at position 155:
    target_residue = None
    for residue in model.get_residues():
        if residue.get_full_id()[3][1] == residue_position and residue.get_resname() == "ASP":
            target_residue = residue
            break
    
    # However, if target_residue is not found, then raise ERROR:
    if target_residue is None:
        raise ValueError(f"Aspartate residue at position {residue_position} not found in the protein structure.")

    # Now, load the ligand SDF file:
    ligands = Chem.SDMolSupplier(sdf_file)
    print(f"There are {len(ligands)} poses in the SDF file")
    valid_poses = []

    # Check which ligand poses form the salt bridge, and save those who do in valid_poses[]
    for ligand in ligands:
            
        distance, is_true = is_salt_bridge(target_residue, ligand)
        
        if is_true:
                valid_poses.append(ligand)
                ligand.SetProp("SD", str(distance)) #Adding the SD tag to the output SDF file
    
    # Save poses that form the salt bridge
    writer = Chem.SDWriter(output_file)
    
    for ligand in valid_poses:
        writer.write(ligand)
    
    writer.close()

### Step 4: Execute the code by calling the functions and save the output

In [4]:
# Give paths to the input files:

pdb_file = "data/protein_6a93.pdb"
sdf_file = "data/ligands_5ht2a.sdf"
output_file = "data/valid_poses.sdf"
residue_position = 155  # Change this to the desired position

# Execute the code by calling the function and passing the parameters
filter_ligand_poses(pdb_file, sdf_file, output_file, residue_position)

There are 414 poses in the SDF file


### Final Step 5: Analyse the output:

In [5]:
output = Chem.SDMolSupplier("data/valid_poses.sdf")
print(f"There are {len(output)} poses that form salt bridge with the ASP155")

There are 25 poses that form salt bridge with the ASP155
